In [ ]:
# 🚀 QUICK RECOVERY - Run this cell after disconnection!
# This single cell sets up everything and resumes training

print("="*60)
print("🔄 Setting up environment...")
print("="*60)

# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Install dependencies
!pip install -q pyyaml tensorboard thop albumentations pycocotools

# 3. Clone code from GitHub
import os
GITHUB_REPO = "https://github.com/habibour/target_detection_paper.git"

!rm -rf /content/code
!git clone {GITHUB_REPO} /content/code

# Navigate to code directory
if os.path.exists('/content/code/implement'):
    os.chdir('/content/code/implement')
else:
    os.chdir('/content/code')
print(f"✅ Working directory: {os.getcwd()}")

# 4. Setup paths
PROJECT_DIR = "/content/drive/MyDrive/paper/target_detection"
CHECKPOINT_DIR = f"{PROJECT_DIR}/HE_YOLOX_checkpoints"
LOG_DIR = f"{PROJECT_DIR}/HE_YOLOX_logs"
RESULTS_DIR = f"{PROJECT_DIR}/HE_YOLOX_results"

!mkdir -p "{CHECKPOINT_DIR}"
!mkdir -p "{LOG_DIR}"
!mkdir -p "{RESULTS_DIR}"

# 5. Create symbolic links to dataset
!mkdir -p data/VisDrone2019
!ln -sf "{PROJECT_DIR}/VisDrone2019-DET-train" data/VisDrone2019/VisDrone2019-DET-train
!ln -sf "{PROJECT_DIR}/VisDrone2019-DET-val" data/VisDrone2019/VisDrone2019-DET-val
!ln -sf "{PROJECT_DIR}/VisDrone2019-DET-test-dev" data/VisDrone2019/VisDrone2019-DET-test-dev
!ln -sf "{PROJECT_DIR}/VisDrone2019-DET-test-challenge" data/VisDrone2019/VisDrone2019-DET-test-challenge

# Verify dataset
print("\n📊 Dataset:")
for split in ['train', 'val', 'test-dev']:
    path = f"data/VisDrone2019/VisDrone2019-DET-{split}/images"
    if os.path.exists(path):
        print(f"  ✅ {split}: {len(os.listdir(path))} images")
    else:
        print(f"  ❌ {split}: Not found")

# 6. Create Colab config
import yaml
with open('configs/he_yolox_asff.yaml', 'r') as f:
    config = yaml.safe_load(f)

config['data']['data_dir'] = './data/VisDrone2019'
config['output']['model_dir'] = CHECKPOINT_DIR
config['output']['log_dir'] = LOG_DIR
config['output']['results_dir'] = RESULTS_DIR
config['train']['batch_size'] = 16
config['val']['batch_size'] = 16
config['train']['num_workers'] = 2
config['val']['num_workers'] = 2
config['train']['save_interval'] = 5  # Save every 5 epochs

with open('configs/he_yolox_asff_colab.yaml', 'w') as f:
    yaml.dump(config, f)

# 7. Find checkpoint and resume training
import glob
import re

checkpoints = glob.glob(f"{CHECKPOINT_DIR}/epoch_*.pth")
resume_cmd = ""

if checkpoints:
    epoch_nums = []
    for ckpt in checkpoints:
        match = re.search(r'epoch_(\d+)\.pth', ckpt)
        if match:
            epoch_nums.append((int(match.group(1)), ckpt))
    
    if epoch_nums:
        epoch_nums.sort(reverse=True)
        last_epoch, latest_ckpt = epoch_nums[0]
        print(f"\n✅ Found checkpoint: epoch {last_epoch}")
        print(f"   Remaining: {300 - last_epoch} epochs")
        resume_cmd = f"--resume {latest_ckpt}"
else:
    print("\n📝 No checkpoint found - starting fresh training")

print("\n" + "="*60)
print("🚀 Starting training...")
print("="*60)

!python train.py \
    --config configs/he_yolox_asff_colab.yaml \
    --device cuda \
    --epochs 300 \
    {resume_cmd}

---
## 📋 Step-by-Step Setup (First Time Only)
Run cells below only for initial setup or debugging.

In [ ]:
# Check GPU
!nvidia-smi

import torch
print(f"\nPyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
# Install dependencies
!pip install -q pyyaml tensorboard thop albumentations pycocotools
print("✅ Dependencies installed")

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Verify dataset
import os
PROJECT_DIR = "/content/drive/MyDrive/paper/target_detection"

print("\n📊 Checking dataset:")
for name in ['VisDrone2019-DET-train', 'VisDrone2019-DET-val', 'VisDrone2019-DET-test-dev']:
    path = f"{PROJECT_DIR}/{name}/images"
    if os.path.exists(path):
        print(f"  ✅ {name}: {len(os.listdir(path))} images")
    else:
        print(f"  ❌ {name}: NOT FOUND")

In [ ]:
# Clone code from GitHub
import os
GITHUB_REPO = "https://github.com/habibour/target_detection_paper.git"

!rm -rf /content/code
!git clone {GITHUB_REPO} /content/code

if os.path.exists('/content/code/implement'):
    os.chdir('/content/code/implement')
else:
    os.chdir('/content/code')

print(f"\n📁 Working directory: {os.getcwd()}")
!ls -la

In [ ]:
# Setup dataset symbolic links
import os
PROJECT_DIR = "/content/drive/MyDrive/paper/target_detection"

!mkdir -p data/VisDrone2019
!ln -sf "{PROJECT_DIR}/VisDrone2019-DET-train" data/VisDrone2019/VisDrone2019-DET-train
!ln -sf "{PROJECT_DIR}/VisDrone2019-DET-val" data/VisDrone2019/VisDrone2019-DET-val
!ln -sf "{PROJECT_DIR}/VisDrone2019-DET-test-dev" data/VisDrone2019/VisDrone2019-DET-test-dev
!ln -sf "{PROJECT_DIR}/VisDrone2019-DET-test-challenge" data/VisDrone2019/VisDrone2019-DET-test-challenge

print("✅ Dataset linked")
!ls -la data/VisDrone2019/

In [ ]:
# Create Colab configuration
import yaml

PROJECT_DIR = "/content/drive/MyDrive/paper/target_detection"
CHECKPOINT_DIR = f"{PROJECT_DIR}/HE_YOLOX_checkpoints"
LOG_DIR = f"{PROJECT_DIR}/HE_YOLOX_logs"
RESULTS_DIR = f"{PROJECT_DIR}/HE_YOLOX_results"

!mkdir -p "{CHECKPOINT_DIR}"
!mkdir -p "{LOG_DIR}"
!mkdir -p "{RESULTS_DIR}"

with open('configs/he_yolox_asff.yaml', 'r') as f:
    config = yaml.safe_load(f)

config['data']['data_dir'] = './data/VisDrone2019'
config['output']['model_dir'] = CHECKPOINT_DIR
config['output']['log_dir'] = LOG_DIR
config['output']['results_dir'] = RESULTS_DIR
config['train']['batch_size'] = 16
config['val']['batch_size'] = 16
config['train']['num_workers'] = 2
config['val']['num_workers'] = 2
config['train']['save_interval'] = 5

with open('configs/he_yolox_asff_colab.yaml', 'w') as f:
    yaml.dump(config, f)

print("✅ Config created")
print(f"   Checkpoints: {CHECKPOINT_DIR}")
print(f"   Batch size: 16")
print(f"   Save every: 5 epochs")

In [ ]:
# Test model
import torch
from models import build_he_yolox

print("Building HE-YOLOX-S...")
model = build_he_yolox("s", num_classes=13).cuda()

x = torch.randn(2, 3, 640, 640).cuda()
with torch.no_grad():
    out = model(x)

params = sum(p.numel() for p in model.parameters()) / 1e6
print(f"✅ Model OK! Output: {out.shape}, Params: {params:.2f}M")

del model, x
torch.cuda.empty_cache()

In [ ]:
# Start training (with auto-resume)
import os
import glob
import re

PROJECT_DIR = "/content/drive/MyDrive/paper/target_detection"
CHECKPOINT_DIR = f"{PROJECT_DIR}/HE_YOLOX_checkpoints"

# Find latest checkpoint
checkpoints = glob.glob(f"{CHECKPOINT_DIR}/epoch_*.pth")
resume_cmd = ""

if checkpoints:
    epoch_nums = []
    for ckpt in checkpoints:
        match = re.search(r'epoch_(\d+)\.pth', ckpt)
        if match:
            epoch_nums.append((int(match.group(1)), ckpt))
    
    if epoch_nums:
        epoch_nums.sort(reverse=True)
        last_epoch, latest_ckpt = epoch_nums[0]
        print(f"✅ Resuming from epoch {last_epoch}")
        resume_cmd = f"--resume {latest_ckpt}"
else:
    print("📝 Starting fresh training")

!python train.py \
    --config configs/he_yolox_asff_colab.yaml \
    --device cuda \
    --epochs 300 \
    {resume_cmd}

---
## 📊 Monitoring & Evaluation

In [ ]:
# TensorBoard
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/paper/target_detection/HE_YOLOX_logs

In [ ]:
# Check GPU usage
!nvidia-smi

In [ ]:
# Evaluate model
import os
import glob

PROJECT_DIR = "/content/drive/MyDrive/paper/target_detection"
CHECKPOINT_DIR = f"{PROJECT_DIR}/HE_YOLOX_checkpoints"

# Find best weights
best = f"{CHECKPOINT_DIR}/best.pth"
if os.path.exists(best):
    weights = best
else:
    ckpts = sorted(glob.glob(f"{CHECKPOINT_DIR}/epoch_*.pth"))
    weights = ckpts[-1] if ckpts else None

if weights:
    print(f"Using: {os.path.basename(weights)}")
    !python eval.py \
        --config configs/he_yolox_asff_colab.yaml \
        --weights {weights} \
        --split val \
        --device cuda
else:
    print("❌ No weights found")

In [ ]:
# Run inference on sample images
import os
import glob
import random
import shutil

PROJECT_DIR = "/content/drive/MyDrive/paper/target_detection"
CHECKPOINT_DIR = f"{PROJECT_DIR}/HE_YOLOX_checkpoints"
RESULTS_DIR = f"{PROJECT_DIR}/HE_YOLOX_results"

# Get sample images
!mkdir -p test_images
val_imgs = glob.glob('data/VisDrone2019/VisDrone2019-DET-val/images/*.jpg')[:5]
for img in val_imgs:
    shutil.copy(img, 'test_images/')

# Find weights
best = f"{CHECKPOINT_DIR}/best.pth"
weights = best if os.path.exists(best) else sorted(glob.glob(f"{CHECKPOINT_DIR}/epoch_*.pth"))[-1]

!python inference.py \
    --config configs/he_yolox_asff_colab.yaml \
    --weights {weights} \
    --source test_images/ \
    --output {RESULTS_DIR}/inference \
    --conf_thresh 0.3 \
    --save_img \
    --device cuda

print(f"\n✅ Results saved to: {RESULTS_DIR}/inference")

In [ ]:
# Display inference results
import matplotlib.pyplot as plt
from PIL import Image
import glob

RESULTS_DIR = "/content/drive/MyDrive/paper/target_detection/HE_YOLOX_results"
images = glob.glob(f'{RESULTS_DIR}/inference/*.jpg') + glob.glob(f'{RESULTS_DIR}/inference/*.png')

if images:
    fig, axes = plt.subplots(1, min(3, len(images)), figsize=(15, 5))
    if len(images) == 1:
        axes = [axes]
    for ax, img_path in zip(axes, images[:3]):
        ax.imshow(Image.open(img_path))
        ax.axis('off')
    plt.tight_layout()
    plt.show()
else:
    print("No results found")

---
## 📁 File Structure

```
/content/drive/MyDrive/paper/target_detection/
├── VisDrone2019-DET-train/      # Dataset (your upload)
├── VisDrone2019-DET-val/
├── VisDrone2019-DET-test-dev/
├── HE_YOLOX_checkpoints/        # Model weights
│   ├── best.pth
│   └── epoch_*.pth
├── HE_YOLOX_logs/               # TensorBoard
└── HE_YOLOX_results/            # Inference output
```

## 🔄 After Disconnection
**Just run Cell 2** - it does everything automatically!

## ⏱️ Training Time
~12-18 hours on T4 GPU for 300 epochs